In [1]:
from dash import Dash,html,dcc
import plotly.express as px
import pandas as pd
import numpy as np
from dash.dependencies import Input,Output,State
import dash_bootstrap_components as dbc

# insta_general

In [2]:
df = pd.read_csv('insta_general.csv', header=0)

In [3]:
df['posts']= df['posts'].astype('string').str.replace(',','').astype('int')

In [4]:
df['followers']= df['followers'].astype('int')
df['followings']= df['followings'].astype('int')

In [5]:
df0 = df.sort_values(by=['followers'])
followers_Bar = px.bar(df0 , y= 'name', x='followers', orientation='h')

In [6]:
df0 = df.sort_values(by=['posts'])
postsNum_Bar = px.bar(df0 , x= 'name', y='posts')

In [7]:
categories_followers_pie = px.pie(df, names='category', values='followers',  color_discrete_sequence= px.colors.sequential.Purpor, title = "Most Followed Category")

In [8]:
tier_pie = px.pie(df, names='tier')

In [9]:
df0 = df.sort_values(by=['avg_engagement_rate'])
er_Bar = px.bar(df0 , x= 'username', y='avg_engagement_rate')

# insta_detailed

In [10]:
df_detailed = pd.read_csv('insta_detailed.csv', header=0)

In [11]:
df_temp = df_detailed.drop(df_detailed[df_detailed['likes']=='unknown'].index)
df_detailed['likes'] = df_detailed['likes'].replace('unknown',df_temp['likes'].astype('int').min())
df_detailed['likes'] = df_detailed['likes'].astype('int')
df_detailed['captions']=df_detailed['captions'].fillna(' ')

In [12]:
posts_by_type_pie = px.pie(df_detailed, names='type')

In [13]:
df_detailed['date'] = pd.to_datetime(df_detailed['date'])

In [14]:
Likes_scatter = px.scatter(df_detailed,
                           x='date',y='likes',
                           color='username',
                           hover_name='username',
                           width=1000,height=1000,
                           title='Date VS Likes',
                           symbol='username',
                           template='seaborn')

# Dash App

In [15]:
#header
card_content0 = [html.H1("Young Content Creators on Instagram", \
         style ={'font-size': '500%', 'text-align': 'center', \
                 "background": "linear-gradient(to top right, ##F7DAE8,  #FEE3D0)", 'height': '50%'})]

In [16]:
def CustomCard(*args, **kwargs):
    print(kwargs)
    option = kwargs['color']
    default_color = "red"
    default_color_light = ""
    default_color_dark = ""
    if option =='primary':
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to top right, orange, deeppink)"})
    elif option =='secondary':
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to bottom left, orange, deeppink)"})
    elif option =='info':
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to top left, orange, deeppink)"})
    else:
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to bottom right, orange, deeppink)"})
    return dbc.Card(*args, **kwargs)

In [17]:
#Big Numbers
card_content1 = [ dbc.CardHeader("Combined Followers"),
                 dbc.CardBody([html.H5(str(df['followers'].sum()), className="card-title")], style ={'text-align': 'center'}),]
card_content2 = [ dbc.CardHeader("Total Posts"),
                 dbc.CardBody([html.H5(str(df['posts'].sum()), className="card-title")], style ={'text-align': 'center'}),]
card_content3 = [ dbc.CardHeader("Total Likes"),
                 dbc.CardBody([html.H5(str(df_detailed['likes'].sum()), className="card-title")], style ={'text-align': 'center'}),]
card_content4 = [ dbc.CardHeader("Highest Engagement Rate"),
                 dbc.CardBody([html.H5(str(round(df['avg_engagement_rate'].max(),2)), className="card-title")], style ={'text-align': 'center'}),]

In [46]:
#photo scroll
photos = dbc.Col([
             dbc.Card([
                dbc.Row([
                    dbc.Col([
                        dbc.CardImg(src=f"/assets/insta.png", top=True,style={"width":"120px", 'clip-path': 'circle(42%)','cursor': 'pointer'}),
                        dbc.CardBody(html.P("Instagram", style = {'text-align': 'center'}))]) ,
                *[
                    dbc.Col([
                        dbc.CardImg(src=f"/assets/{i}.jpg", top=True,style={"width":"120px", 'clip-path': 'circle(42%)', 'cursor': 'pointer'}),
                        dbc.CardBody(html.P(df.username[i], style = {'text-align': 'center'}))]) for i in range(df.shape[0])]
                ], style={"flexWrap":"nowrap" }),
            ], style={'overflowX': 'scroll',  })
        ], style={"display": "flex", 'padding': '5px'})

In [42]:
# pie + scatter
card_content5 = [dcc.Graph(figure=categories_followers_pie), ]
                
card_content6 = [dcc.Graph(id='graph4'),
     dcc.Dropdown(options=[{'label': str(username), 'value': str(username)} \
                                for username in df_detailed['username'].unique()],id='ddown3',multi=True,)]

In [20]:
# Bars
card_content7 = dcc.Graph(id='graph1')
card_content8 = dcc.Graph(id='graph2')
card_content9 = dcc.Graph(id='graph3')

card_content10 =[ dcc.Dropdown(options=[{'label': str(category), 'value': str(category)} \
                          for category in df['category'].unique()], id='ddown1',
                ),
                 dcc.Dropdown(options=[{'label': str(tier), 'value': str(tier)} \
                                       for tier in df['tier'].unique()], id='ddown2',) ]

In [53]:
#Discover Posts
card_content_posts = [ html.H2("Discover Posts",style = {'text-align': 'center', 'color': 'pink'}),
                 dbc.CardBody([dbc.Card(id='card1'),
     dcc.Dropdown(options=[{'label': str(username), 'value': str(username)} \
                                for username in df['username'].unique()],id='ddown4',),
     dcc.Dropdown(id='ddown5',),]),]


In [ ]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

@app.callback(
    Output(component_id='graph1',component_property='figure'),
    Output(component_id='graph2',component_property='figure'),
    Output(component_id='graph3',component_property='figure'),
    Input(component_id='ddown1',component_property='value'),
    Input(component_id='ddown2',component_property='value'),
)
def update_My_Div(category,tier):
    if category==None and tier==None:
        df0 = df.sort_values(by=['followers'] , ascending=False)
        df1 = df.sort_values(by=['posts'], ascending=False)
        df2 = df.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'username', y='followers', title = "Who is most followed?", color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        postsNum_Bar = px.bar(df1 , x= 'username', y='posts', title = "Who posts the most?", color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate' , title = "Who has the highest Engagement Rate?" , color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        return followers_Bar,postsNum_Bar, er_Bar
    elif tier==None:
        df0 = df[df['category'] == category]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'username', y='followers', title = "Who is most followed?" , color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        postsNum_Bar = px.bar(df1 , x= 'username', y='posts' , title = "Who posts the most?", color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate', title = "Who has the highest Engagement Rate?", color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        return followers_Bar,postsNum_Bar, er_Bar
    elif category==None:
        df0 = df[df['tier'] == tier]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'username', y='followers', title = "Who is most followed?", color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        postsNum_Bar = px.bar(df1 , x= 'username', y='posts' , title = "Who posts the most?", color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate', title = "Who has the highest Engagement Rate?", color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        return followers_Bar,postsNum_Bar,er_Bar
        
    else:
        df0 = df[np.logical_and(df['category'] == category,df['tier'] == tier)]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'username', y='followers', title = "Who is most followed?", color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        postsNum_Bar = px.bar(df1 , x= 'username', y='posts' , title = "Who posts the most?", color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate', title = "Who has the highest Engagement Rate?", color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        return followers_Bar,postsNum_Bar, er_Bar
    
@app.callback(
    Output(component_id='graph4',component_property='figure'),
    Input(component_id='ddown3',component_property='value'),
)
def update_My_Div(username):
    if username==None or len(username)==0:
        Likes_scatter = px.scatter(df_detailed,
                        x='date',y='likes',
                        color='username',
                        hover_name='username',
                        title='Date VS Likes',
                        symbol='username',
                        color_discrete_sequence= px.colors.sequential.Purpor)
        return Likes_scatter
    else:
        df_scatter = df_detailed[np.isin(df_detailed.username,username)]
        Likes_scatter = px.scatter(df_scatter,
                        x='date',y='likes',
                        color='username',
                        hover_name='username',
                        title='Date VS Likes',
                        symbol='username',
                        color_discrete_sequence= px.colors.sequential.Purpor)
        return Likes_scatter

@app.callback(
    Output(component_id='ddown5',component_property='options'),
    Input(component_id='ddown4',component_property='value'),
)

def update_My_Div(username):
    df_name = df_detailed[df_detailed['username']==username]
    options=[{'label': str(date), 'value': str(date)}  for date in df_name['date'].unique()]

    return options

@app.callback(
    Output(component_id='card1',component_property='children'),
    Input(component_id='ddown4',component_property='value'),
    Input(component_id='ddown5',component_property='value'),
)

def update_My_Div(username, date):
    
    df_name = df_detailed[df_detailed['username']==username]
    
    if username == None:
        children = []
    elif date == None or ((df_name['date']!=date).all()) :
        children = [
                    dbc.CardBody([html.H4(username, className="card-title"),]),
                   ]
    else:
        children = [
                    dbc.CardBody([html.H4(username, className="card-title"),]),
                    html.P(df_name[df_name['date'] == date].iloc[0,5], className="card-text",),
                    dbc.Button("Open Post", color="primary", href=df_name[df_name['date'] == date].iloc[0,1],),
                   ]

    return children
    
    


app.layout =html.Div(children=[
    
 html.Div(
    [
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content0)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(CustomCard(card_content1, color="primary", inverse=True )),
                dbc.Col(CustomCard(card_content2, color="secondary", inverse=True)),
                dbc.Col(CustomCard(card_content3, color="info", inverse=True)),
                dbc.Col(CustomCard(card_content4, color="success", inverse=True)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(dbc.Card(photos)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(card_content5, width = 4),
                dbc.Col(card_content_posts , width=3),
                dbc.Col(card_content6, width =5),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(card_content7,  width=4),
                dbc.Col(card_content8,  width=4),
                dbc.Col(card_content9,  width=4),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(card_content10),
            ],
            className="mb-4",
        ),
    ]
)
    ])
app.run_server()

{'color': 'primary', 'inverse': True}
{'color': 'secondary', 'inverse': True}
{'color': 'info', 'inverse': True}
{'color': 'success', 'inverse': True}
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" 

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Apr/2022 23:03:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 23:03:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 23:03:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 23:03:26] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 23:03:26] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 23:03:26] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 23:03:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 23:03:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 23:03:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 23:03:27] "GET /_favicon.ico?v=2.3.0 HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 23:03:27] "POST /_dash-update-component HTTP/1.1" 